In [81]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
import pickle

# Remove man from original dataset

In [35]:
original_df = pd.read_csv('../raw_data/okcupid_profiles.csv')

In [102]:
original_df.age.unique()

array([ 22,  35,  38,  23,  29,  32,  31,  24,  37,  28,  30,  39,  33,
        26,  27,  20,  25,  40,  36,  21,  34,  43,  46,  41,  42,  45,
        18,  55,  50,  59,  44,  48,  54,  51,  62,  52,  19,  58,  66,
        53,  63,  47,  49,  61,  60,  57,  56,  65,  64,  68, 110,  69,
        67, 109])

In [38]:
female_df = original_df[original_df['sex'] == 'f']

In [104]:
female_df.to_csv('../raw_data/ok_female.csv', columns=female_df.columns, index=False)

In [99]:
female_df[female_df['age'] < 49]

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
6,32,single,f,straight,fit,strictly anything,socially,never,graduated from college/university,"white, other",...,life is about the little things. i love to lau...,digging up buried treasure,frolicking witty banter using my camera to ext...,i am the last unicorn,i like books. ones with pictures. reading them...,laughter amazing people in my life color curio...,synchronicity there is this whole other realm...,plotting to take over the world with my army o...,my typical friday night,NaN
7,31,single,f,straight,average,mostly anything,socially,never,graduated from college/university,white,...,NaN,"writing. meeting new people, spending time wit...","remembering people's birthdays, sending cards,...",i'm rather approachable (a byproduct of being ...,"i like: alphabetized lists, aquariums, autobio...","friends, family, notebook/pen, books, music, t...",things that amuse and inspire me,out and about or relaxing at home with a good ...,NaN,NaN
8,24,single,f,straight,NaN,strictly anything,socially,NaN,graduated from college/university,white,...,NaN,"oh goodness. at the moment i have 4 jobs, so i...",NaN,i'm freakishly blonde and have the same name a...,i am always willing to try new foods and am no...,sports/my softball glove coffee. because nobod...,NaN,"in or out... drinking with friends, maybe a ba...",potential friends/lovers/people who come in co...,http://www.youtube.com/watch?v=4dxbwzuwsxk let...
13,30,single,f,straight,skinny,mostly anything,socially,never,graduated from high school,white,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,29,single,f,straight,thin,mostly anything,socially,never,working on college/university,"hispanic / latin, white",...,NaN,i have an awesome career working as a senior m...,listening. helping others. being patient. comm...,"well, i get the most compliments on my butt, s...",books: my all-time favorite book is george orw...,1. my family 2. italian or mexican food 3. my ...,"to be honest, i spend way too much time thinki...",unwinding from my work week. hanging out with ...,"hmmm...not sure if i want to admit this, but p...",...you genuinely think we'd be a good match.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59933,30,single,f,straight,NaN,NaN,socially,never,NaN,white,...,i'm a southern girl from tennesse who moved to...,i'm in human resources for a luxury retail com...,listening and giving advice!,my smile :-),"i like any and all types of music, especially ...","wine, college football, sweet tea, family, fri...",it's not glamorous but...work.,out for drinks or dinner with friends!,embarrassing fact-i love being organized...and...,you want to meet someone new :)
59934,43,single,f,straight,average,anything,socially,NaN,graduated from masters program,white,...,i'm an east coast/west coast hybrid who's live...,"starting a business; raising a wonderful, sill...","finding joy in simple things, getting lost, ma...",i've been told my eyes but also that i smile a...,i love reading but these days more of my time ...,"1. water....in the ocean, lake, bathtub 2. sen...","-how to be a better parent, friend, daughter, ...","home with my son, playing with legos, wrestlin...","i'm recently divorced, but have been separated...",-you are intrigued by any of the above -you ar...
59935,33,single,f,straight,curvy,anything,socially,never,NaN,white,...,i am a mom first and foremost. currently separ...,i currently work at safeway. hopefully not for...,a lot of things...,my eyes and then the fact that i am pretty str...,i love to read and so i do not have an absolut...,my kids my ipod/iphone water my car my mac my ...,everything i have and want to do!,"either working, hang in' with my kids or out h...",hahaha! i'm not telling...yet.,"you kept reading after currently separated, ar..."
59936,25,single,f,straight,average,mostly anything,

# LDA 

In [2]:
df = pd.read_csv('../raw_data/combined_text_preprocessed.csv',keep_default_na=False)

#### Get just female combined preprocessed

In [43]:
female_combined_preprocessed_df = df [df.index.isin(female_df.index)]

In [41]:
df

,combined_preprocessed,answered_percent,text_length
0,would love think kind intellectual either dumb...,100.0,1565
1,chef mean workaholic love cook regardless whet...,60.0,815
2,im ashamed much write public text online date ...,90.0,3728
3,work library go school read thing write old de...,70.0,330
4,hey hows go currently vague profile know come ...,50.0,496
...,...,...,...
59941,vibrant expressive care optimist love people t...,100.0,1040
59942,im nick never know write im sure hand im south...,100.0,1634
59943,hello enjoy travel watch movie hang friend rul...,100.0,1189
59944,world ball integrity one take either away momm...,100.0,1122


In [44]:
female_combined_preprocessed_df

,combined_preprocessed,answered_percent,text_length
6,life little thing love laugh easy one find bea...,90.0,1061
7,write meet new people spend time friend see fi...,70.0,935
8,oh goodness moment job itd nice find one could...,70.0,1436
13,,0.0,0
14,awesome career work senior manager large nonpr...,90.0,1614
...,...,...,...
59934,im east coastwest coast hybrid who live studie...,100.0,2321
59935,mom first foremost currently separate file div...,100.0,687
59936,infectious laugh,10.0,16
59940,real deal sensitive sorry tear include sillyse...,100.0,3475


## Finding two topics

### Vectorizing

In [47]:
%%time
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(min_df=0.1, ngram_range=(1,2))

# Training it on the texts
combined_weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(female_combined_preprocessed_df.combined_preprocessed.values).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

combined_weighted_words

CPU times: user 10 s, sys: 802 ms, total: 10.8 s
Wall time: 11.3 s


,actually,adventure,almost,along,also,always,amaze,american,anything,appreciate,...,word,work,world,would,write,year,yes,yet,yoga,youre
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.110815,0.0,0.000000,0.115610,...,0.000000,0.053256,0.160778,0.074450,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.000000,0.091499,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.076020,0.000000,0.000000,0.262736,0.000000,0.0,0.000000,0.171546,0.000000
2,0.000000,0.000000,0.0,0.000000,0.000000,0.074841,0.000000,0.0,0.000000,0.000000,...,0.000000,0.051739,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.084910
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.092077,0.000000,0.0,0.000000,0.163855,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.136135,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.074471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24112,0.000000,0.060901,0.0,0.079019,0.042309,0.000000,0.146289,0.0,0.139717,0.076309,...,0.000000,0.000000,0.053061,0.000000,0.000000,0.141858,0.0,0.000000,0.079323,0.000000
24113,0.000000,0.000000,0.0,0.000000,0.309575,0.000000,0.000000,0.0,0.085192,0.000000,...,0.000000,0.128602,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
24114,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
24115,0.000000,0.000000,0.0,0.062954,0.000000,0.081019,0.000000,0.0,0.000000,0.121591,...,0.060254,0.028005,0.042274,0.117453,0.048395,0.075345,0.0,0.186914,0.000000,0.091920


In [50]:
tf_idf_vectorizer

TfidfVectorizer(min_df=0.1, ngram_range=(1, 2))

### Save vectorizer as pickle

In [48]:
# save
with open('../pkl/tf_idf_vectorizer.pkl','wb') as f:
    pickle.dump(tf_idf_vectorizer,f)

### Load vectorizer

In [49]:
# load
with open('../pkl/tf_idf_vectorizer.pkl', 'rb') as f:
    tfidf_vec = pickle.load(f)

In [51]:
tfidf_vec

TfidfVectorizer(min_df=0.1, ngram_range=(1, 2))

### Training LDA

In [52]:
%%time

# Instantiate the LDA
n_components = 2
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter=100, random_state=42)

# Fit the LDA on the vectorized documents
lda_model.fit(combined_weighted_words)

CPU times: user 4min 38s, sys: 7.23 ms, total: 4min 38s
Wall time: 4min 38s


LatentDirichletAllocation(max_iter=100, n_components=2, random_state=42)

### Document topic mixture

In [53]:
%%time
document_topic_mixture = lda_model.transform(combined_weighted_words)

document_topic_mixture

CPU times: user 1.92 s, sys: 0 ns, total: 1.92 s
Wall time: 2 s


array([[0.62453782, 0.37546218],
       [0.29951184, 0.70048816],
       [0.74415463, 0.25584537],
       ...,
       [0.26911743, 0.73088257],
       [0.6189887 , 0.3810113 ],
       [0.42579334, 0.57420666]])

In [56]:
two_topics_df = pd.DataFrame({
    'topic_0_from_two': document_topic_mixture[:, 0],
    'topic_1_from_two': document_topic_mixture[:, 1],
    'original_text': female_combined_preprocessed_df.combined_preprocessed
})

In [57]:
two_topics_df

,topic_0_from_two,topic_1_from_two,original_text
6,0.624538,0.375462,life little thing love laugh easy one find bea...
7,0.299512,0.700488,write meet new people spend time friend see fi...
8,0.744155,0.255845,oh goodness moment job itd nice find one could...
13,0.500000,0.500000,
14,0.562000,0.438000,awesome career work senior manager large nonpr...
...,...,...,...
59934,0.523496,0.476504,im east coastwest coast hybrid who live studie...
59935,0.643762,0.356238,mom first foremost currently separate file div...
59936,0.269117,0.730883,infectious laugh
59940,0.618989,0.381011,real deal sensitive sorry tear include sillyse...


### Topic means

In [58]:
document_topic_matrix = two_topics_df.iloc[:, :2]
document_topic_matrix

,topic_0_from_two,topic_1_from_two
6,0.624538,0.375462
7,0.299512,0.700488
8,0.744155,0.255845
13,0.500000,0.500000
14,0.562000,0.438000
...,...,...
59934,0.523496,0.476504
59935,0.643762,0.356238
59936,0.269117,0.730883
59940,0.618989,0.381011


In [59]:
topic_distribution = np.sum(document_topic_matrix, axis=0)
topic_distribution

topic_0_from_two    13017.820006
topic_1_from_two    11099.179994
dtype: float64

In [60]:
# Normalize to get proportions
topic_proportions = topic_distribution / np.sum(topic_distribution)
topic_proportions

topic_0_from_two    0.539778
topic_1_from_two    0.460222
dtype: float64

### Most relevant words per topic

In [61]:
topic_word_mixture = pd.DataFrame(
    lda_model.components_,
    columns = tf_idf_vectorizer.get_feature_names_out()
)

topic_word_mixture

,actually,adventure,almost,along,also,always,amaze,american,anything,appreciate,...,word,work,world,would,write,year,yes,yet,yoga,youre
0,346.941823,55.404964,229.898013,183.934931,738.306701,438.745487,145.600327,190.114608,617.970686,108.410566,...,322.371352,536.999757,369.913875,528.753963,609.047604,371.934169,305.281908,256.20948,1.226435,675.414175
1,1.228142,508.548801,104.108708,79.768703,371.892242,306.618323,198.176597,101.198491,300.599237,201.055839,...,0.827631,937.599913,335.323376,296.068794,1.348369,495.112656,1.150939,24.48686,353.491412,17.243021


In [91]:
topic_word_mixture.iloc[1].sort_values(ascending = False).head(10)

love      1882.895873
friend    1603.317134
enjoy     1169.935548
new       1123.874101
family    1114.195666
travel     938.064917
work       937.599913
good       937.386017
music      937.147558
life       935.172512
Name: 1, dtype: float64

### Saving pickle model

In [64]:
# save
with open('../pkl/model.pkl','wb') as f:
    pickle.dump(lda_model,f)

In [65]:
# load
with open('../pkl/model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [66]:
clf2

LatentDirichletAllocation(max_iter=100, n_components=2, random_state=42)

In [67]:
%%time

topic_means = document_topic_matrix.mean()
print(topic_means)

topic_0_from_two    0.539778
topic_1_from_two    0.460222
dtype: float64
CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.62 ms


In [68]:
lda_model

LatentDirichletAllocation(max_iter=100, n_components=2, random_state=42)

### Add text-length

In [69]:
two_topics_df['text_length'] = two_topics_df.combined_preprocessed.str.len()

,topic_0_from_two,topic_1_from_two,original_text
6,0.624538,0.375462,life little thing love laugh easy one find bea...
7,0.299512,0.700488,write meet new people spend time friend see fi...
8,0.744155,0.255845,oh goodness moment job itd nice find one could...
13,0.500000,0.500000,
14,0.562000,0.438000,awesome career work senior manager large nonpr...
...,...,...,...
59934,0.523496,0.476504,im east coastwest coast hybrid who live studie...
59935,0.643762,0.356238,mom first foremost currently separate file div...
59936,0.269117,0.730883,infectious laugh
59940,0.618989,0.381011,real deal sensitive sorry tear include sillyse...


In [70]:
female_combined_preprocessed_df

,combined_preprocessed,answered_percent,text_length
6,life little thing love laugh easy one find bea...,90.0,1061
7,write meet new people spend time friend see fi...,70.0,935
8,oh goodness moment job itd nice find one could...,70.0,1436
13,,0.0,0
14,awesome career work senior manager large nonpr...,90.0,1614
...,...,...,...
59934,im east coastwest coast hybrid who live studie...,100.0,2321
59935,mom first foremost currently separate file div...,100.0,687
59936,infectious laugh,10.0,16
59940,real deal sensitive sorry tear include sillyse...,100.0,3475


In [78]:
nina_df = pd.concat([female_combined_preprocessed_df, two_topics_df], axis=1).drop(columns=['answered_percent', 'original_text'])

In [82]:
upper_threshold = nina_df['text_length'].quantile(0.99)
nina_df['text_length_clipped'] = nina_df['text_length'].clip(upper=upper_threshold)
scaler = MinMaxScaler()
nina_df['text_length_scaled'] = scaler.fit_transform(nina_df[['text_length_clipped']])

In [83]:
nina_df

,combined_preprocessed,text_length,topic_0_from_two,topic_1_from_two,text_length_clipped,text_length_scaled
6,life little thing love laugh easy one find bea...,1061,0.624538,0.375462,1061.0,0.219077
7,write meet new people spend time friend see fi...,935,0.299512,0.700488,935.0,0.193061
8,oh goodness moment job itd nice find one could...,1436,0.744155,0.255845,1436.0,0.296508
13,,0,0.500000,0.500000,0.0,0.000000
14,awesome career work senior manager large nonpr...,1614,0.562000,0.438000,1614.0,0.333262
...,...,...,...,...,...,...
59934,im east coastwest coast hybrid who live studie...,2321,0.523496,0.476504,2321.0,0.479244
59935,mom first foremost currently separate file div...,687,0.643762,0.356238,687.0,0.141853
59936,infectious laugh,16,0.269117,0.730883,16.0,0.003304
59940,real deal sensitive sorry tear include sillyse...,3475,0.618989,0.381011,3475.0,0.717525


### Save MinMaxScaler pickle

In [85]:
# save
with open('../pkl/text_length_scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [92]:
nina_df = nina_df.drop(columns=['combined_preprocessed', 'text_length', 'text_length_clipped'])

In [93]:
nina_df.to_csv('../raw_data/nina_dataframe.csv', columns=nina_df.columns, index=False)

In [94]:
pd.read_csv('../raw_data/nina_dataframe.csv')

,topic_0_from_two,topic_1_from_two,text_length_scaled
0,0.624538,0.375462,0.219077
1,0.299512,0.700488,0.193061
2,0.744155,0.255845,0.296508
3,0.500000,0.500000,0.000000
4,0.562000,0.438000,0.333262
...,...,...,...
24112,0.523496,0.476504,0.479244
24113,0.643762,0.356238,0.141853
24114,0.269117,0.730883,0.003304
24115,0.618989,0.381011,0.717525


In [96]:
imgs_df = pd.read_csv('../raw_data/output_images_ages_adjusted.csv')

In [97]:
imgs_df[]

,image_path,gender,age
0,data/1 (25).jpeg,female,(18-20)
1,data/1 (5).jpg,male,(21-34)
2,data/man1.jpg,male,(21-34)
3,data/woman1.jpg,female,(50-60)
4,data/minion.jpg,male,(50-60)
...,...,...,...
466,data/1 (279).jpg,male,(21-34)
467,data/1 (372).jpg,male,(21-34)
468,data/1 (335).jpg,male,(18-20)
469,data/1 (251).jpg,male,(18-20)


In [105]:
ok = pd.read_csv('../raw_data/ok_stream.csv')

In [107]:
ok

,female,age_scaled,single,height_scaled,orientation_bisexual,orientation_gay,orientation_straight,education_type_college_univ,education_type_grad_or_professional_edu,education_type_not_disclosed,...,speaks_english,speaks_spanish,speaks_other,diet_type_vegetarian,has_dogs_yes,no_of_kids_more_than_one,no_of_kids_one,text_length_scaled,topic_0_from_two,bio
0,1,0.274510,1,0.346154,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0.214859,0.317310,"32 years old woman, currently single. Zodiac s..."
1,1,0.254902,1,0.346154,0,0,1,1,0,0,...,1,1,0,0,0,0,0,0.191053,0.747795,"31 years old woman, currently single. Zodiac s..."
2,1,0.117647,1,0.423077,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0.292127,0.295034,"24 years old woman, currently single. Zodiac s..."
3,1,0.235294,1,0.384615,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0.007868,0.500000,"30 years old woman, currently single. Has a sk..."
4,1,0.215686,1,0.230769,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0.326424,0.439788,"29 years old woman, currently single. Zodiac s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24112,1,0.490196,1,0.269231,0,0,1,0,1,0,...,1,0,0,0,1,0,1,0.468250,0.491193,"43 years old woman, currently single. Zodiac s..."
24113,1,0.294118,1,0.423077,0,0,1,0,0,1,...,1,0,0,0,0,1,0,0.138599,0.465117,"33 years old woman, currently single. Zodiac s..."
24114,1,0.137255,1,0.192308,0,0,1,1,0,0,...,1,0,1,0,0,0,0,0.010491,0.738342,"25 years old woman, currently single. Zodiac s..."
24115,1,0.254902,1,0.230769,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0.701064,0.391134,"31 years old woman, currently single. Zodiac s..."


In [108]:
imgs = pd.read_csv('../raw_data/output_images_ages_adjusted.csv')

In [120]:
female_images_df = imgs[imgs['gender'] == 'female']

In [121]:
female_images_df

,image_path,gender,age
0,data/1 (25).jpeg,female,(18-20)
3,data/woman1.jpg,female,(50-60)
5,data/1 (28).jpeg,female,(50-60)
8,data/girl1.jpg,female,(21-34)
15,data/1 (3).jpeg,female,(35-49)
...,...,...,...
455,data/1 (322).jpg,female,(21-34)
458,data/1 (268).jpg,female,(21-34)
459,data/1 (350).jpg,female,(21-34)
463,data/1 (339).jpg,female,(21-34)


In [122]:
female_images_df.to_csv('../raw_data/female_images_df.csv', index=False, columns=female_images_df.columns)

In [124]:
female_images_df[female_images_df['image_path'] == 'data/1 (50).jpeg']

,image_path,gender,age
165,data/1 (50).jpeg,female,(18-20)
172,data/1 (50).jpeg,female,(18-20)
179,data/1 (50).jpeg,female,(18-20)
186,data/1 (50).jpeg,female,(18-20)
